In [2]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")  # notebooks/ 기준 상위
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("✅ sys.path 추가 완료:", PROJECT_ROOT)

✅ sys.path 추가 완료: c:\Users\mungm\Documents\ai_engineer\genai_rag2


In [3]:
import pprint
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 중요: src를 명시적으로 기준으로 사용
from src.vectorstore.chroma_client import get_chroma_collection
from src.processing.embedder import embed_text  # bge-m3

pp = pprint.PrettyPrinter(indent=2, width=120)

collection = get_chroma_collection()
print("✅ Chroma collection 로드 완료")


c:\Users\mungm\Documents\ai_engineer\genai_rag2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> USING DB PATH = c:\Users\mungm\Documents\ai_engineer\genai_rag2\src\data\chroma_db
✅ Chroma collection 로드 완료


In [4]:
from src.vectorstore.chroma_client import get_chroma_collection
import pprint

collection = get_chroma_collection()

results = collection.get(
    where={"notice_id": "20240330003"},
    include=["documents", "metadatas", "embeddings"],  # ✅ ids 넣지 말기
)

# ✅ chunk_id는 include가 아니라 results["ids"]로 봅니다
print("chunk_id 개수:", len(results["ids"]))
print("첫 chunk_id:", results["ids"][0] if results["ids"] else None)

pprint.pp(results)

>>> USING DB PATH = c:\Users\mungm\Documents\ai_engineer\genai_rag2\src\data\chroma_db
chunk_id 개수: 268
첫 chunk_id: 189ca543-9d7e-4cb3-8828-0752220ef7e7
{'ids': ['189ca543-9d7e-4cb3-8828-0752220ef7e7',
         '0060d1f2-1855-4de1-9ad8-f1b543611436',
         'fbcdf3b9-c610-4bb7-b83a-f3fc87e58c9b',
         '307799ee-2f67-4e39-9e80-5d1dd6f304a4',
         '9a83f6ce-df78-404f-b39a-d2ee45df2b9e',
         '0188c2b1-f282-439b-9561-4e427006bbd8',
         'cc132681-4987-459a-9424-bc6c3aef74ee',
         '0ffd6781-8c18-49c3-a3b4-841170c602b3',
         '8e02e016-c388-4b69-bb96-4b1c15d24309',
         '816ebc1f-94fd-4a60-96c9-a6d5a89c15b9',
         'a1555fdc-0669-4adb-80c5-c3500f047456',
         '2266b8bd-7ee7-4d45-befe-d82dea77142c',
         '8f8c3054-4236-4c87-9d46-02eb46408b99',
         'a8469554-c5d2-4158-aaef-f1bddeeb6962',
         '2ce595f8-98a5-42bc-85cf-afbedd2a540e',
         '18540f59-36ec-4a50-af73-aa88cb66b13c',
         '8d247dbd-10e5-4b00-8183-b3ca44c01bb2',
         '7e44

In [6]:
docs = results.get("documents", [])
metas = results.get("metadatas", [])
embs = results.get("embeddings", [])
ids = results.get("ids", [])

for i in range(min(len(ids), 5)):  # 우선 5개만
    print("\n" + "=" * 90)
    print(f"[{i}] chunk_id:", ids[i])
    print("page:", metas[i].get("page") if i < len(metas) else None)
    print("section:", metas[i].get("section") if i < len(metas) else None)
    print("text(head):", (docs[i][:200] + "...") if i < len(docs) else None)

    if i < len(embs) and embs[i] is not None:
        print("embedding_dim:", len(embs[i]))
        print("embedding_sample:", embs[i][:10])
    else:
        print("embedding: None (저장 안 됐거나 include 누락)")


[0] chunk_id: 189ca543-9d7e-4cb3-8828-0752220ef7e7
page: 1
section: None
text(head): 2222222222222000000000000022222222222224444444444444년년년년년년년년년년년년년 ｢｢｢｢｢｢｢｢｢｢｢｢｢벤벤벤벤벤벤벤벤벤벤벤벤벤처처처처처처처처처처처처처확확확확확확확확확확확확확인인인인인인인인인인인인인종종종종종종종종종종종종종합합합합합합합합합합합합합관관관관관관관관관관관관관리리리리리리리리리리리리리시시시시시시시시시시시시시스스스스...
embedding_dim: 1024
embedding_sample: [-0.05176677  0.04561433 -0.00338273 -0.01952172 -0.02998227 -0.03266806
 -0.01486036  0.00375062 -0.0063511   0.01302619]

[1] chunk_id: 0060d1f2-1855-4de1-9ad8-f1b543611436
page: 2
section: None
text(head): 목 차 1. 추진개요 3 2. 추진방안 5 3. 추진내용 9 4. 제안요청내용 24 5. 입찰관련사항 78 6. 제안서작성요령 82 7. 별지서식 및 붙임 94...
embedding_dim: 1024
embedding_sample: [-0.02639146 -0.00060405  0.01407853  0.00500255  0.00234935 -0.00410384
  0.00902663  0.00160477  0.0038244  -0.01284471]

[2] chunk_id: fbcdf3b9-c610-4bb7-b83a-f3fc87e58c9b
page: 3
section: Ⅰ. 추진개요
text(head): Ⅰ. 추진개요 1 추진배경 및 방향 □ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵 션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이 관, 신규 구축업무를 

In [1]:
from src.vectorstore.chroma_client import get_chroma_collection
from src.processing.embedder import embed_text
import pprint

collection = get_chroma_collection()

NOTICE_ID = "20240330003"

results = collection.get(
    where={"notice_id": NOTICE_ID},
    include=["documents", "metadatas", "embeddings"],  # ❗ ids는 넣지 않음
)

docs = results.get("documents", [])
metas = results.get("metadatas", [])
embs = results.get("embeddings", [])
ids = results.get("ids", [])

print(f"\n📌 notice_id = {NOTICE_ID}")
print(f"총 chunk 수: {len(ids)}")

for i in range(min(len(ids), 5)):  # 우선 5개만
    print("\n" + "=" * 90)
    print(f"[{i}] chunk_id:", ids[i])

    meta = metas[i] if i < len(metas) else {}
    print("page:", meta.get("page"))
    print("section:", meta.get("section"))
    print("subsection:", meta.get("subsection"))
    print("source_file:", meta.get("source_file"))

    if i < len(docs):
        print("text(head):", docs[i][:200] + "...")

    if i < len(embs) and embs[i] is not None:
        print("embedding_dim:", len(embs[i]))
        print("embedding_sample:", embs[i][:10])
    else:
        print("embedding: None")

ModuleNotFoundError: No module named 'src'

In [24]:
from src.vectorstore.chroma_client import get_chroma_collection

collection = get_chroma_collection()

sample = collection.get(limit=10, include=["metadatas"])

# 메타 키 수집
keys = set()
for meta in sample["metadatas"]:
    keys.update(meta.keys())

print("📌 Metadata schema (keys):")
for k in sorted(keys):
    print("-", k)

>>> USING DB PATH = c:\Users\mungm\Documents\ai_engineer\genai_rag2\src\data\chroma_db
📌 Metadata schema (keys):
